# Mass Spectra Preprocessing
How is the mass spectra converted so that it can be used in  CNN or GNN model.

## Summary
We now have the mass spectra extracted from the MGF files where they are stored in a single file. With the spectra_locations file mapping where the beginning and start of individual spectra are.

I am sure this entire process can be replaced with Pyteomics. We will have to discuss the process with the team.

I know how to:
1. Read and process an MGF file to obtain the mass spectra
2. Create a spectrum location file and save it as npy file.
3. Create a dataset containing the scan, peptide_ids, spectrum, spectrum intensity, peptide mass and charge.

## 1. Input process of the mass spectra
The first step is to view how the raw MS file is used as input into the model.

### 1.1. Input Parameters
To train a model, edit the path to the training and validation files and run the following:
```
python run_graph_model.py --logging_dir <training_directory> --org <organism_type>
```

So the first file is ```run_graph_model.py```:
It has two parameters:
1. logging_dir the directory where the logs will be stored
2. org: the organism type

The input parameters are handled by the ```model_utils.arg``` function. The ```model_utils``` hold the configurations of the model.

But only two input parameters are used: ```logging_dir``` and ```org```.

In [109]:
import os
import re
import numpy as np
from pathlib import Path
BASEDIR = Path(os.path.abspath('')).parent
print(BASEDIR)
DATADIR = os.path.join(BASEDIR, 'data')
print(DATADIR)



/home/wesley/instadeep/codebase/MSencoding
/home/wesley/instadeep/codebase/MSencoding/data


We will set these parameters in the next cell:

In [11]:
logging_dir = "graphs/" + 'logs'
print(logging_dir)

graphs/logs


We next create a log file to store the logs in the current directory we are working in. If the file does not exist, we create it. We can then open the file in append mode:

In [12]:
log_file = os.getcwd() + "/" + logging_dir + "/LogFile.txt"
if not os.path.exists(os.path.dirname(log_file)):
    os.makedirs(os.path.dirname(log_file))
log_file_handle = open(log_file, "a")


### 1.2. Input Data
The next step is to import the data. Here two ```data_types``` are specified by the ```model_utils.py``` configuration file:
1. "msp"
2. "mgf" - Mascot Generic Format DenovoV1 Species dataset.

The input mass spectrum files are opened and the mass spectra is processed with the ```read_spectra_only``` function.

In [27]:
data_path = Path.joinpath(Path(DATADIR), 'sample_data', 'sample_preprocessed_spectra.mgf')
print(data_path)

/home/wesley/instadeep/codebase/MSencoding/data/sample_data/sample_preprocessed_spectra.mgf


In [79]:
# config
mass_H = 1.0078
mass_N_terminus = 1.0078
mass_C_terminus = 17.0027
mass_H2O = 18.0106

MZ_MAX = 3000
PRECURSOR_MASS_PRECISION_INPUT_FILTER = 1000
mass_AA = {'_PAD': 0.0,
           '_GO': mass_N_terminus - mass_H,
           '_EOS': mass_C_terminus + mass_H,
           'A': 71.03711,  # 0
           'R': 156.10111,  # 1
           'N': 114.04293,  # 2
           'Nmod': 115.02695,
           'D': 115.02694,  # 3
           # 'C': 103.00919, # 4
           'Cmod': 160.03065,  # C(+57.02)
           # ~ 'Cmod':161.01919, # C(+58.01) # orbi
           'E': 129.04259,  # 5
           'Q': 128.05858,  # 6
           'Qmod': 129.0426,
           'G': 57.02146,  # 7
           'H': 137.05891,  # 8
           'I': 113.08406,  # 9
           'L': 113.08406,  # 10
           'K': 128.09496,  # 11
           'M': 131.04049,  # 12
           'Mmod': 147.0354,
           'F': 147.06841,  # 13
           'P': 97.05276,  # 14
           'S': 87.03203,  # 15
           'T': 101.04768,  # 16
           'W': 186.07931,  # 17
           'Y': 163.06333,  # 18
           'V': 99.06841,  # 19
           }



In [101]:
########################################################################
# VOCABULARY
########################################################################


_PAD = "_PAD"
_GO = "_GO"
_EOS = "_EOS"
_START_VOCAB = [_PAD, _GO, _EOS]

PAD_ID = 0
GO_ID = 1
EOS_ID = 2

vocab_reverse = ['A',
                 'R',
                 'N',
                 'Nmod',
                 'D',
                 # 'C',
                 'Cmod',
                 'E',
                 'Q',
                 'Qmod',
                 'G',
                 'H',
                 'I',
                 'L',
                 'K',
                 'M',
                 'Mmod',
                 'F',
                 'P',
                 'S',
                 'T',
                 'W',
                 'Y',
                 'V',
                 ]

vocab_reverse = _START_VOCAB + vocab_reverse

vocab = dict([(x, y) for (y, x) in enumerate(vocab_reverse)])

vocab_size = len(vocab_reverse)

In [105]:

def read_spectra_only(file_handle, spectra_locations, data_format='mgf'):
    """
    Read the spectra from the file handle and return the spectra

    """
    global scan, peptide_mass, charge, peptide_len, spectrum_mz, peptide
    data_set = []

    max_seq_len = 32 - 2 # why -2?

    counter = 0
    counter_skipped = 0
    counter_skipped_mod = 0
    counter_skipped_len = 0
    counter_skipped_mass = 0
    counter_skipped_mass_precision = 0

    avg_peak_count = 0.0
    avg_peptide_len = 0.0

    # This block reads the MGF file
    keyword = None
    if data_format == 'mgf':
        keyword = 'BEGIN IONS'
    # the spectra file contains multiple spectra, this moves the file reader to the correct location for a new spectrum.
    # There must be a way to generate file spectrum locations.
    for location in spectra_locations.values():
        file_handle.seek(location)
        line = file_handle.readline()
        assert (keyword in line), "Error: MGF file is not in the correct format"
        unknown_modification = False

        # Read the spectra.mgf file,
        if data_format == 'mgf':
            line = file_handle.readline()
            title = line
            # Pepmass
            line = file_handle.readline()
            peptide_ion_mz = float(re.split('=|\n', line)[1])

            # Charge
            line = file_handle.readline()
            charge = float(re.split('=|\+', line)[1])

            # Scans
            line = file_handle.readline()
            scan = re.split('=|\n', line)[1]

            # RTINSECONDS
            line = file_handle.readline()
            rtinseconds = line

            # Annotated Sequence can have modifications
            line = file_handle.readline()
            sequence = re.split('=|\n|\r', line)[1]
            sequence_len = len(sequence)
            peptide = []
            index = 0
            # logger.info("Implemented modifications are Cmod, Mmod, Nmod, Qmod")
            # this block of code iterates through the annotated sequence and checks for a modification. If a modification (the modified mass) it is replaced by the modification in the vocab
            # If no modification is found then it continues to append
            while index < sequence_len:
                if sequence[index] == "(" or sequence[index]=='+':
                    if peptide[-1] == "C" and sequence[index: index + 8] == "(+57.02)" or sequence[index: index + 7] == "+57.021":
                        peptide[-1] = "Cmod"
                        index += 8
                    elif peptide[-1] == "M" and sequence[index: index + 8] == "(+15.99)" or sequence[index: index + 7] == "+15.994":
                        peptide[-1] = "Mmod"
                        index += 8
                    elif peptide[-1] == 'N' and sequence[index: index + 6] == "(+.98)" or sequence[index: index + 5] == "+.984":
                        peptide[-1] = "Nmod"
                        index += 6
                    elif peptide[-1] == 'Q' and sequence[index: index + 6] == "(+.98)" or sequence[index: index + 5] == "+.984":
                        peptide[-1] = "Qmod"
                        index += 6

                    else:
                        unknown_modification = True
                        break
                else:
                    peptide.append(sequence[index])
                    index += 1

            if unknown_modification:
                counter_skipped_mod += 1
                counter_skipped += 1
                continue

            # peptide mass is calculated from parent mass X charge X mass of H - Cterm - Nterm
            # this is mass of the parent ion. I am not sure why we drop the C and N-term from the mass
            peptide_mass = (peptide_ion_mz * charge * mass_H) - mass_N_terminus - mass_C_terminus

            # we only look at peptides with a mass less than a maximum value
            if peptide_mass > MZ_MAX:
                counter_skipped_mass += 1
                counter_skipped += 1
                continue

            peptide_len = len(peptide)
            # we only look at peptides with a length less than a maximum value
            if peptide_len > max_seq_len:
                counter_skipped_len += 1
                counter_skipped += 1
                continue
            # we calculate the mass of the sequence
            sequence_mass = sum(mass_AA[aa] for aa in peptide)

            # if the sequence and peptide mass are not the same then we skip the sequence
            if abs(peptide_mass - sequence_mass) > PRECURSOR_MASS_PRECISION_INPUT_FILTER:
                counter_skipped_mass_precision += 1
                counter_skipped += 1
                continue

            # next we read the peaks and their intensities
            spectrum_mz = []
            spectrum_intensity = []
            line = file_handle.readline()
            while not "END IONS" in line:
                mz, intensity = line.split(' ')[0], line.split(' ')[1]
                intensity_float = float(intensity)
                mz_float = float(mz)

                if mz_float > MZ_MAX:
                    line = file_handle.readline() # stops the reading if mz is greater than the max
                    continue

                spectrum_mz.append(mz_float)
                spectrum_intensity.append(intensity_float)

                line = file_handle.readline()
        counter += 1

        peak_count = len(spectrum_mz)
        avg_peak_count += peak_count
        avg_peptide_len += peptide_len
        peptide_ids = [vocab[aa] for aa in peptide]

        data_set.append([scan,
                         peptide_ids,
                         spectrum_mz,
                         peptide_mass,
                         charge])

    return data_set





A missing function is how spectrum locations are determined. I found the following code in PointNovo

In [95]:
def get_spectrum_locations(spectrum_file: Path):
    spectrum_location_dict = {}
    line = True
    with open(spectrum_file, 'r') as f:
        while line:
            current_location = f.tell()
            line = f.readline()
            if "BEGIN IONS" in line:
                spectrum_location = current_location
            elif "SCANS=" in line:
                scan = line.split('=')[1].strip()
                spectrum_location_dict[scan] = spectrum_location
    return spectrum_location_dict

PointNovo saves the spectrum location in a pickle file while this implementation saves it a npy file.

In [87]:
spectrum_loc = get_spectrum_locations(data_path)
save_path = Path.joinpath(data_path.parent, 'spectrum_locations.npy')
np.save(save_path, spectrum_loc)

In [106]:
with open(data_path) as file_handle:
    spectra = read_spectra_only(file_handle, spectra_locations=spectrum_loc, data_format='mgf')

In [107]:
spectra

[['F1:2478',
  [14, 3, 13, 24, 5, 16, 4],
  [63.994834899902344,
   70.06543731689453,
   84.081298828125,
   85.08439636230469,
   86.09666442871094,
   110.07109069824219,
   129.1020050048828,
   138.06597900390625,
   157.13291931152344,
   175.1185302734375,
   185.1283721923828,
   209.10263061523438,
   273.1337890625,
   301.1282958984375,
   303.21221923828125,
   304.17529296875,
   322.1859130859375,
   350.6787414550781,
   417.2552185058594,
   580.3185424804688,
   630.36572265625,
   717.376708984375,
   753.3748779296875,
   788.4207763671875,
   866.4544677734375],
  891.536014288,
  2.0],
 ['F1:2484',
  [25, 16, 9, 7, 20, 7, 12, 9, 13, 3, 4],
  [107.2381362915039,
   110.07106018066406,
   127.08580780029297,
   129.10214233398438,
   133.4121551513672,
   151.06536865234375,
   175.11810302734375,
   183.14926147460938,
   225.36390686035156,
   228.1688232421875,
   239.09396362304688,
   240.13259887695312,
   255.14370727539062,
   286.1373291015625,
   339.200653

## Summary
We now have the mass spectra extracted from the MGF files where they are stored in a single file. With the spectra_locations file mapping where the beginning and start of individual spectra are.

I am sure this entire process can be replaced with Pyteomics. We will have to discuss the process with the team.

I know how to:
1. Read and process an MGF file to obtain the mass spectra
2. Create a spectrum location file and save it as npy file.
3. Create a dataset containing the scan, peptide_ids, spectrum, spectrum intensity, peptide mass and charge.